In [2]:
import wrds
import pandas as pd
import numpy as np
import os

# Get the current working directory
script_dir = os.getcwd()

# Set the working directory to the current script's directory (which in this case is already the working directory)
os.chdir(script_dir)

print(f"Working directory is set to: {script_dir}")

# Connect to WRDS
db = wrds.Connection(wrds_username='zrsong')

# Define the start and end dates
start_date = '1994-01-01'
end_date = '2024-06-30'

'''
# Use S&P ratings data as Compustat ratings data were discontinued in 2017
# Query the S&P ratings data
query = f"""
    SELECT gvkey, ratingdate, ratingsymbol
    FROM ciq_ratings.wrds_erating 
    WHERE ratingdate >= '{start_date}' AND ratingdate <= '{end_date}'
"""

# Execute the query and fetch the data
ratings_date = db.raw_sql(query)

# Save the data to a CSV file
ratings_date.to_csv('../Data/ratings_data.csv', index=False)
'''

Working directory is set to: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/Code
Loading library list...
Done


'\n# Use S&P ratings data as Compustat ratings data were discontinued in 2017\n# Query the S&P ratings data\nquery = f"""\n    SELECT gvkey, ratingdate, ratingsymbol\n    FROM ciq_ratings.wrds_erating \n    WHERE ratingdate >= \'{start_date}\' AND ratingdate <= \'{end_date}\'\n"""\n\n# Execute the query and fetch the data\nratings_date = db.raw_sql(query)\n\n# Save the data to a CSV file\nratings_date.to_csv(\'../Data/ratings_data.csv\', index=False)\n'

In [10]:
# show available libraries from wrds
libraries = db.list_libraries()

# show available tables from wrds
tables = db.list_tables(library='comp_na_daily_all')
print(tables)

['aco_amda', 'aco_imda', 'aco_indfnta', 'aco_indfntq', 'aco_indfntytd', 'aco_indsta', 'aco_indstq', 'aco_indstytd', 'aco_notesa', 'aco_notesq', 'aco_notessa', 'aco_notesytd', 'aco_pnfnda', 'aco_pnfndq', 'aco_pnfndytd', 'aco_pnfnta', 'aco_pnfntq', 'aco_pnfntytd', 'aco_transa', 'aco_transq', 'aco_transsa', 'aco_transytd', 'adsprate', 'asec_amda', 'asec_imda', 'asec_notesa', 'asec_notesq', 'asec_transa', 'asec_transq', 'chars', 'co_aacctchg', 'co_aaudit', 'co_adesind', 'co_adjfact', 'co_afnd1', 'co_afnd2', 'co_afnddc1', 'co_afnddc2', 'co_afntind1', 'co_afntind2', 'co_ainvval', 'co_amkt', 'co_busdescl', 'co_cotype', 'co_filedate', 'co_fortune', 'co_hgic', 'co_iacctchg', 'co_iaudit', 'co_idesind', 'co_ifndq', 'co_ifndsa', 'co_ifndytd', 'co_ifntq', 'co_ifntsa', 'co_ifntytd', 'co_imkt', 'co_industry', 'co_ipcd', 'co_mthly', 'co_offtitl', 'company', 'currency', 'dd_group', 'dd_group_xref', 'dd_item', 'dd_package', 'ecind_desc', 'ecind_mth', 'exrt_dly', 'exrt_mth', 'funda', 'funda_fncd', 'fundq

In [35]:
# Annual Compustat data
fund_table = 'funda'
# use the varlist above to query quarterly compustat data (don't use join(varlist) as it will return a string)
query = f"""
    SELECT *
    FROM comp_na_daily_all.{fund_table}
    WHERE datadate >= '{start_date}' AND datadate <= '{end_date}'
"""
compa = db.raw_sql(query)

OperationalError: (psycopg2.OperationalError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: 
    SELECT *
    FROM comp_na_daily_all.funda
    WHERE datadate >= '1994-01-01' AND datadate <= '2024-06-30'
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [15]:
# all colnames of compa
# check if sic exists
# for each gvkey fyear, keep the one with the highest at
compa = compa.sort_values(['gvkey', 'fyear', 'at'], ascending=[True, True, False])
compa = compa.drop_duplicates(subset=['gvkey', 'fyear'], keep='first')
# output csv. format
compa.to_csv("../Data/Raw/compustat_annual.csv", index=False)

In [25]:

# Quarterly Compustat data
fund_table = 'fundq'
varlist = ['gvkey', 'conm', 'tic', 'cusip','fyearq', 'fqtr', 'fyr', 'atq','capxy', 'ceqq', 'cogsq', 
           'cshoq', 'dlcq', 'dlcchy','dlttq', 'dpq', 'ibq', 'itccy', 'fic', 'pstkrq',
           'ltq', 'mibq', 'niq', 'prstkccy', 'pstkq', 'req', 'revtq', 'saleq',
           'seqq', 'txdbq', 'txdiq', 'txditcq', 'wcapchy', 'xinty', 'xrdq', 'xrdy', 'xsgaq',
           'mkvaltq', 'epspxq', 'epsfxq', 'ajexq', 'prccq', 'oancfy', 'ivncfy', 'rdq', 'ppegtq', 'ppentq']
# Join the list into a comma-separated string
varlist_str = ", ".join(varlist)
# use the varlist above to query quarterly compustat data (don't use join(varlist) as it will return a string)
query = f"""
    SELECT {varlist_str}
    FROM comp_na_daily_all.{fund_table}
    WHERE datadate >= '{start_date}' AND datadate <= '{end_date}'
"""
compq = db.raw_sql(query)

In [26]:
# check for years in the data
compq['fyearq'].unique()

array([2006, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2007, 2008, 2010, 2009, 2016, 2017, 2011, 2012,
       2013, 2014, 2015, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [27]:
# How to deal with duplicates? Keep last available entry (datadate)
compq.dropna(subset=['fyearq', 'fqtr'], inplace=True)
compq.sort_values(['gvkey','fyearq', 'fqtr', 'atq'], inplace=True)
compq = compq[~compq.duplicated(['gvkey', 'fyearq', 'fqtr'], keep='last')]

In [28]:
duplicate_columns = compq.columns[compq.columns.duplicated()]
print("Duplicate columns:", duplicate_columns)

Duplicate columns: Index([], dtype='object')


In [29]:
# Calculating BE
# Shareholder Equity
compq['se'] = compq['seqq']
# Uses Common Equity (ceq) + Preferred Stock (pstk) if SEQ is missing:
compq['se'].fillna((compq['ceqq'] + compq['pstkq']), inplace=True)
# Uses Total Assets (at) - Liabilities (lt) + Minority Interest (mib, if available), if others are missing
compq['se'].fillna((compq['atq'] - compq['ltq'] + compq['mibq'].fillna(0)), inplace=True)

/var/folders/n2/pgdf58ms1yd3ch8xjn9fmpmm0000gq/T/ipykernel_35699/4278593784.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  compq['se'].fillna((compq['ceqq'] + compq['pstkq']), inplace=True)
/var/folders/n2/pgdf58ms1yd3ch8xjn9fmpmm0000gq/T/ipykernel_35699/4278593784.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are s

In [30]:
# Preferred Stock
# Preferred Stock (Redemption Value)
compq['ps'] = compq['pstkrq']
# Uses Preferred Stock (Liquidating Value (pstkl)) if Preferred Stock (Redemption Value) is missing (pstkl doesnt' exist for quarterly data)
# compq['ps'].fillna(compq['pstkl'], inplace=True)
# Uses Preferred Stock (Carrying Value (pstk)) if others are missing
compq['ps'].fillna(compq['pstkq'], inplace=True)

/var/folders/n2/pgdf58ms1yd3ch8xjn9fmpmm0000gq/T/ipykernel_35699/4121065189.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  compq['ps'].fillna(compq['pstkq'], inplace=True)


In [31]:
# Deferred Taxes
# Uses Deferred Taxes and Investment Tax Credit (txditc)
compq['dt'] = compq['txditcq']
# This was Novy-Marx old legacy code. We drop this part to be in accordance with Ken French.
# Uses Deferred Taxes and Investment Tax Credit(txdb) + Investment Tax Credit (Balance Sheet) (itcb) if txditc is missing (itcb (Investment Tax Credit (Balance Sheet)) doesn't exist for quarterly data)
compq['dt'].fillna(compq['txdbq'].fillna(0), inplace=True)
# If all measures are missing, set to missing
compq.loc[pd.isnull(compq['txditcq']) & pd.isnull(compq['txdbq']), 'dt'] = np.nan

/var/folders/n2/pgdf58ms1yd3ch8xjn9fmpmm0000gq/T/ipykernel_35699/1696288597.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  compq['dt'].fillna(compq['txdbq'].fillna(0), inplace=True)


In [32]:
# Book Equity
# Book Equity (BE) = Share Equity (se) - Prefered Stocks (ps) + Deferred Taxes (dt)
compq['be'] = (compq['se']  # shareholder equity must be available, otherwise BE is missing
               - compq['ps']  # preferred stock must be available, otherwise BE is missing
               + compq['dt'].fillna(0))  # add deferred taxes if available
               #- compa['prba'].fillna(0))  # subtract postretirement benefit assets if available

In [33]:
# COST = COGS + XSGA + XINT 
compq['cost'] = compq[['cogsq', 'xsgaq', 'xinty']].sum(axis=1, skipna=True)
compq.loc[compq[['cogsq', 'xsgaq', 'xinty']].isnull().all(axis=1), 'cost'] = np.nan
# OP = SALE - COST    
compq['op'] = compq['saleq']-compq['cost']

In [34]:
# output csv. format
compq.to_csv("../Data/Raw/compustat_quarterly.csv", index=False)

In [ ]:
# Query the dealscan syndicated loan data
query = f"""
    SELECT *
    FROM tr_dealscan.dealscan
    WHERE deal_active_date >= '{start_date}' AND deal_active_date <= '{end_date}'
"""

# Execute the query and fetch the data
dealscan_data = db.raw_sql(query)
# Save the data to a CSV file
dealscan_data.to_csv('../Data/Raw/dealscan_data.csv', index=False)

In [3]:
# obtain Dealscan legacy lenders data (to use 2018 JF bank name link)
query = f"""
    SELECT *
    FROM tr_dealscan.lendershares
"""
lendershares = db.raw_sql(query)

# output csv. format
lendershares.to_csv("../Data/Raw/lendershares.csv", index=False)

In [4]:
# Disconnect from WRDS
db.close()